In [1]:
import numpy as np
import pandas as pd

from requests_oauthlib import OAuth1

import pprint
import json
import time

import requests
import twitter
import tweepy

import data_scientists

In [2]:
from twitter_credentials import credentials

oauth = OAuth1(credentials["TWITTER_CONSUMER_KEY"],
               credentials["TWITTER_CONSUMER_KEY_SECRET"],
               credentials["TWITTER_ACCESS_TOKEN"],
               credentials["TWITTER_ACCESS_TOKEN_SECRET"])

In [3]:
credentials['TWITTER_CONSUMER_KEY']

'4sCoD6VGBWCoxcsGMqfpovVXE'

In [4]:
tweets = requests.get("https://stream.twitter.com/1.1/statuses/filter.json", auth=oauth)

In [5]:
tweets

<Response [406]>

In [3]:
consumer_key = credentials['TWITTER_CONSUMER_KEY']
consumer_secret = credentials['TWITTER_CONSUMER_KEY_SECRET']
access_token_key = credentials['TWITTER_ACCESS_TOKEN']
access_token_secret = credentials['TWITTER_ACCESS_TOKEN_SECRET']

In [7]:
t = twitter.Twitter(auth=twitter.OAuth(access_token_key,access_token_secret,consumer_key,consumer_secret))

In [9]:
ds = t.lists.members(owner_screen_name="stephenilhardt", slug="ds-project-list", count=50)

In [4]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)

In [5]:
auth.set_access_token(access_token_key,access_token_secret)

In [6]:
api = tweepy.API(auth)

printer = pprint.PrettyPrinter()

In [13]:
public_tweets = api.home_timeline()

In [15]:
for tweet in public_tweets:
    print(tweet.text)

Great to hear. Which university do you teach at? https://t.co/mf0dHdEdbg
@David_S_Wilson Kudos. And bravo to @nature for reviewing it.
@neontaster It’s important work
Woof https://t.co/1KbPWcHsqn
RT @anirvan: A Pair of Christian Colleges Quietly Cracking Down on LGBTQ Acceptance https://t.co/5V4M7BoHKX
RT @DavidRutz: Warren at 9 percent in New Hampshire Democrats poll, behind Biden (28), Sanders (20) and Harris (14) https://t.co/OMikLxcYWE
RT @Ciquah: I found the @MSNBC @Morning_Joe segment with @MichaelEMann and @dwallacewells. Definitely a kinda bittersweet feeling about it.…
The DNC’s debate rules won’t make the 2020 primaries any less chaotic. https://t.co/sokLs7QZut
Did I need to know this? No, I did not. https://t.co/zniH025S1y
What Are Your Thoughts: Quadruple Top or Record Highs? https://t.co/12Cihs36EN
Entertaining and informative. It is like having a phone that works when folded and unfolded. https://t.co/cCrMDCDaB1
Wait that sounds like a good thing tho https://t.co/PKQRNppv

In [40]:
cursor = tweepy.Cursor(api.home_timeline,tweet_mode='extended')

In [45]:
cursor.items().next().full_text

'What’s her “lane,” now that Bernie is in? If Warren is the answer, what’s the question? https://t.co/1KbPWcHsqn'

In [ ]:
seed_list = []

In [97]:
users = api.list_members('stephenilhardt','ds-project-list')

In [15]:
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.TweepError:
            time.sleep(15 * 60)
            continue
        except StopIteration:
            break

In [206]:
ds_accounts = pd.DataFrame(columns=['name','id'])

In [7]:
api = data_scientists.get_api()

In [8]:
seed_list = data_scientists.get_seed_list()

In [9]:
len(seed_list)

97

In [11]:


type(tweepy.Cursor(api.list_members, 'stephenilhardt','data-scientists',tweet_mode='extended'))

tweepy.cursor.Cursor

In [13]:
for user in seed_list:
    print(user.friends_count)

905
1487
953
48
4434
2117
2585
751
417
716
1112
132
8
134
1658
394
873
544
507
819
98
175
189
633
4060
1460
283
599
948
378
3402
210
1051
362
574
157
168
1454
1613
1971
485
1447
159
1999
135
1428
564
625
1357
886
1180
2290
741
410
2135
3159
296
1774
287
398
1985
278
196
551
855
1087
2256
1148
20
1237
11
438
1720
1886
1167
1083
1130
2470
198
2946
4306
917
143
115
1281
221
828
559
129
4538
1317
244
240
1576
2340
888
173


In [14]:
len(seed_list)

97

In [10]:
ds_accounts = data_scientists.get_data_scientists(api, seed_list)

TypeError: 'NoneType' object is not iterable

In [18]:
ds_accounts = pd.DataFrame(columns=['name','id','counter'])

In [17]:


for user in seed_list:
    for friends_id in limit_handled(tweepy.Cursor(api.friends_ids, user._json['id']).items()):
        friend_frame = pd.DataFrame()

        friend_frame['name'] = user._json['name']
        friend_frame['id'] = friends_id
        friend_frame['counter'] = 1

        ds_accounts.append(friend_frame)

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [21]:
for friends_id in tweepy.Cursor(api.friends_ids, seed_list[0]._json['id']).items():
    friend_frame = pd.DataFrame()

    friend_frame['name'] = user._json['name']
    friend_frame['id'] = friends_id
    friend_frame['counter'] = 1

    ds_accounts.append(friend_frame)

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [158]:
for scientist in seed_list:
    following = api.friends_ids(scientist._json['id'], tweet_mode='extended')
    following_frame = pd.DataFrame(data=following, columns=['id'])
    ds_accounts.append(following_frame)

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [185]:
friends = []
for friend in tweepy.Cursor(api.friends_ids, seed_list[0]._json['id']).items():
    friends.append(friend)

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [188]:
friends

[]

In [ ]:
ds_accounts['followers'] = [api.get_user(account)._json['followers'] for account in ds_accounts['id']]

In [146]:
for user in tweepy.Cursor(api.list_members, 'stephenilhardt', 'ds-project-list',tweet_mode='extended').items():
    friends = api.friends_ids(user._json['id'], tweet_mode='extended')
    friends_frame = pd.DataFrame(data=friends, columns=['id'])
    data_scientists['id'].append(friends_frame)

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [147]:
len(data_scientists)

0

In [148]:
for user in tweepy.Cursor(api.list_members, 'stephenilhardt', 'ds-project-list',tweet_mode='extended').items():
    print(user._json['name'],user._json['id'])

Dr. GP Pulipaka 4263007693
Evan Sinar, PhD 1024244078
Lillian Pierson, PE 894057794
Jonathan Frye 822338022
Ronald van Loon #MWC19 555031989
Kirk Borne 534563976
Ferenc Huszár🇪🇺 438301250
Mike Tamir, PhD 331659804
Sebastian Thrun 318063815
Kenneth Cukier 250103509
Andrew Ng 216939636
Kathleen Walch 175759038
Mark Tabladillo 129315638
Carla Gentry 119802433
Wes McKinney 115494880
Christopher D. Long 77810367
Hadley Wickham 69133574
Randy Olson 49413866
Gil Press 49383389
Yann LeCun 48008938
Chris Surdak 46370540
Kira Radinsky 38398940
Dr. Craig Brown 36773483
Jeremy 👁🐝Ⓜ️ 36740214
David Smith 34677653
Michael Wu Ph.D. 23221629
Jake Porway 22512172
Amy Heineike 22213177
Bob E. Hayes 20748873
KDnuggets 20167623
Drew Conway 18463930
Ben Lorica 罗瑞卡 18318677
Dean Abbott 17911125
Olivier Grisel 16067035
Jeff Hammerbacher 16061930
Tamara Dull 15612251
John Myles White 15379361
dj patil 14839109
Josh Wills 14578294
Marck Vaisman 14458327
Ronald Schmelzer 14414242
Peter Skomoroch 14344469
Dean of

In [ ]:
for user in tweepy.Cursor(api.list_members, 'stephenilhardt', 'ds-project-list',tweet_mode='extended').items():
    print(user._json['name'],user._json['id'])

In [117]:
api.get_user('20167623')._json['followers_count']

141074

In [93]:
for user in users:
    print(user._json['name'])

Dr. GP Pulipaka
Evan Sinar, PhD
Lillian Pierson, PE
Jonathan Frye
Ronald van Loon #MWC19
Kirk Borne
Ferenc Huszár🇪🇺
Mike Tamir, PhD
Sebastian Thrun
Kenneth Cukier
Andrew Ng
Kathleen Walch
Mark Tabladillo
Carla Gentry
Wes McKinney
Christopher D. Long
Hadley Wickham
Randy Olson
Gil Press
Yann LeCun


In [57]:
ds._json

{'created_at': 'Wed Feb 20 20:33:34 +0000 2019',
 'id': 1098319773010616320,
 'id_str': '1098319773010616320',
 'full_text': 'In mid-Feb Trump signed an executive order to spur the development and regulation of AI including better educating and training workers in the field of AI and cognitive technologies. Are you prepared? @Cognilytica can help. Our next open enrollment… https://t.co/2gLwqLRw6x https://t.co/XIpVXvhsLg',
 'truncated': False,
 'display_text_range': [0, 273],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'cognilytica',
    'name': 'Cognilytica',
    'id': 902566155243671552,
    'id_str': '902566155243671552',
    'indices': [201, 213]}],
  'urls': [{'url': 'https://t.co/2gLwqLRw6x',
    'expanded_url': 'http://dlvr.it/QzKcS1',
    'display_url': 'dlvr.it/QzKcS1',
    'indices': [250, 273]}],
  'media': [{'id': 1098319771135750144,
    'id_str': '1098319771135750144',
    'indices': [274, 297],
    'media_url': 'http://pbs.twimg.com/

In [71]:
api.favorites('bigdatagal', tweet_mode='extended')[0]._json

{'created_at': 'Wed Feb 20 02:00:46 +0000 2019',
 'id': 1098039729709305856,
 'id_str': '1098039729709305856',
 'full_text': 'After the success of the culty salad chain Sweetgreen, kale-scarfing, ginger-quaffing consumers have VCs salivating over salad. The next trend? Fast-casual dining startups—powered by AI and data-mining apps, of course https://t.co/G4Pn6Yilpv',
 'truncated': False,
 'display_text_range': [0, 241],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/G4Pn6Yilpv',
    'expanded_url': 'https://wired.trib.al/xUKLfPs',
    'display_url': 'wired.trib.al/xUKLfPs',
    'indices': [218, 241]}]},
 'source': '<a href="http://www.socialflow.com" rel="nofollow">SocialFlow</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1344951,
  'id_str': '1344951',
  'name': 'WIRED',
  'screen_name': 'WI